# Searching MAAP CMR

A quick way to generate a list of granule URLs based on the CMR Host, DOI, over an Area of Interest (AOI).

In [1]:
cmr = "cmr.maap-project.org"
doi = "10.3334/ORNLDAAC/2056"
aoi = "../tests/fixtures/MONTS_BIROUGOU_NATIONAL_PARK.geojson"
limit = 10

In [2]:
# Converting the bounds of the AOI for maap.searchGranule
import geopandas as gpd

bounds = gpd.read_file(aoi).total_bounds
bbox = ",".join([item for item in bounds.astype(str)])

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/envs/gedi_subset/share/proj failed


In [3]:
from maap.maap import MAAP
from gedi_subset.maapx import find_collection

granules = []
maap = MAAP("api.ops.maap-project.org")

# Search for the collection concept ID using find_collection
for collection in find_collection(maap, cmr, {"doi": f"*{doi}*"}):

    # Search for related granules within the collection over the AOI
    for granule in maap.searchGranule(
        cmr_host=cmr,
        collection_concept_id=collection["concept-id"],
        bounding_box=bbox,
        limit=limit,
    ):
        granules.append(
            next(
                (
                    obj["URL"]
                    for obj in granule["Granule"]["OnlineAccessURLs"]["OnlineAccessURL"]
                    if obj["URL"].startswith("https://")  # https or s3
                ),
                granule["Granule"]["OnlineAccessURLs"]["OnlineAccessURL"][1]["URL"],
            )
        )
granules

['https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019146134206_O02558_01_T05641_02_002_02_V002.h5',
 'https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019160192444_O02779_04_T02282_02_002_02_V002.h5',
 'https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019197172428_O03352_01_T04218_02_002_02_V002.h5',
 'https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019245095256_O04092_04_T04975_02_002_02_V002.h5',
 'https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019290155834_O04794_04_T01976_02_002_02_V002.h5',
 'https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_2019313070136_O05145_04_T00400_02_002_02_V002.h5',
 'https://data.ornldaac.earthdata.nasa.gov/protected/gedi/GEDI_L4A_AGB_Density_V2_1/data/GEDI04_A_20193131

In [ ]:
with open("granules.txt", "w") as f:
    for granule_url in granules:
        f.write(f"{granule_url}\n")